# Classificação de dígitos utilizando descritores de redes neurais e classificador SVM

Veremos um exemplo de uso de uma rede neural para obtenção de descritores de imagens. O exemplo foi feito de forma que não seja necessário treinar a rede em nenhum momento. Para uma aplicação real, é mais comum que o final da rede seja adaptado de forma que não seja necessário o uso do classificador SVM. Mas nesse caso é necessário realizar um treinamento adicional da rede.

Vamos considerar que as imagens a serem utilizadas estão organizadas em diretórios da seguinte forma:

```
digitos  
│
└───treinamento
│   │   digito_0_imagem_0.png
│   │   digito_0_imagem_1.png
│   │   ...
│   │   digito_1_imagem_0.png
│   │   digito_1_imagem_2.png
│   │   ...
│   │   digito_9_imagem_0.png
│   │   digito_9_imagem_1.png
│   │   ...
│  
└───teste
    │   imagem_0.png
    │   imagem_1.png
    │   imagem_2.png
    │   ...
```

O diretório `treinamento` possui 1000 imagens para cada dígito. O diretório `teste` possui algumas imagens cuja classe é desconhecida. Essas imagens serão classificadas após o trainamento do classificador.

Caso você não tenha uma placa de vídeo possuindo núcleos CUDA (placas da NVIDIA), você pode tentar executar esse notebook no Google Colab:

https://colab.research.google.com/drive/1ytJzUos-DKVtnuxuugKFccWOS5UIfgfN?usp=sharing

### Importação de bibliotecas e definição de funções básicas 

In [18]:
import numpy as np                  # (testado na versão 1.17.4)
import sklearn.svm                  # scikit-learn (testado na versão 0.22.1)
from PIL import Image               # Para leitura de imagens (testado na versão 6.2.1)
import os                           # Biblioteca padrão do Python com funções de sistema operacional (versão Python 3.7.4)
import re                           # Biblioteca padrão do Python para expressões regulares
import torch                        # PyTorch (testado na versão 1.6.0)
from torchvision import models      # Modelos de redes neurais para imagens (testado na versão 0.7.0)
from torchvision import transforms  # Transformações de imagens

def filename_to_label(file):
    '''Função que analisa o nome de uma imagem e retorna a classe associada.
    Operações abaixo são específicas para as imagens de dígitos.'''
    
    pattern = re.compile('digito_(\d)')         # Compila expressão regular
    m = pattern.search(file)                    # Busca um match
    return int(m.group(1))                      # Retorna o que está dentro do parênteses na expressão regular

def image_generator(root_folder, files, func_filename_to_label=None):
    '''Gerador de imagens. `func_filename_to_label` é uma função
    que recebe o nome de um arquivo e retorna a classe da imagem.'''
    
    for file in files:
        img = Image.open(root_folder+file)
        if func_filename_to_label is None:
            label = None
        else:
            label = func_filename_to_label(file)
        
        yield img, label

### Leitura das imagens

In [2]:
root_folder = 'digitos/'
train_folder = root_folder + 'treinamento/'

files = os.listdir(train_folder)
imgs = iter(image_generator(train_folder, files, filename_to_label))    # Cria iterador a partir de gerador

In [3]:
# Experimente rodar esta célula diversas vezes usando ctrl+enter
img, label = next(imgs)
img_shape = img.size
print(img_shape)
print(label)
img

(28, 28)
0


### Aplicação da rede neural nas imagens

In [4]:
# Carrega o modelo de rede neural resnet18. Existem inúmeros outros modelos de rede. Esta rede foi pré-treinada
# na base de imagens ImageNet, que possui milhões de imagens, para classificar as imagens em 1000 classes distintas. 
# O comando model.eval() indica que a rede será usada apenas para extração de descritores (não haverá treinamento).
model = models.resnet18(pretrained=True)         
model.eval()

# Para aplicarmos a rede neural na imagem precisamos realizar diversos passos de conversão:
# 1. Obtemos o próximo elemento da base de dados
img, label = next(imgs)

# 2. A rede neural espera uma imagem colorida. Como as nossas imagens são em níveis de cinza,
# convertemos a imagem para RGB. A imagem resultante possui 3 canais idênticos
img = img.convert('RGB')

# 3. É esperado um objeto to tipo torch.Tensor, que se comporta basicamente como um array numpy,
# mas precisamos converter explicitamente para esse tipo. O submódulo `transforms` do PyTorch 
# possui a classe `ToTensor` para conversão de imagens Pillow para tensor. Como ela é uma classe,
# criamos uma instância dela e imediatamente aplicamos na imagem
img = transforms.ToTensor()(img)

# 4. O PyTorch sempre trabalha com batches de imagem. Isto é, ao invés de enviarmos sempre uma única
# imagem para a rede enviamos um conjunto de n imagens simultaneamente. É muito mais eficiente
# processar diversas imagens de uma vez do que cada imagem separadamente. Portanto, a rede neural
# espera sempre um batch de imagens. Um batch possui sempre 4 dimensões, organizadas da seguinte forma
# [tamanho do batch, número de canais, número de linhas, número de colunas]
# Como temos apenas uma imagem, precisamos criar um batch (um array de 4 dimensões) contendo apenas
# uma imagem. Isso é feito pelo método `unsqueeze(0)`, que adiciona uma dimensão no começo do array
img = img.unsqueeze(0)
print(img.shape)

# 5. Finalmente, aplicamos o modelo na imagem
descriptors = model(img)
print(descriptors, descriptors.shape)

# Os comandos acima podem ser chamados em uma única linha
#img = transforms.ToTensor()(img.convert('RGB')).unsqueeze(0)

torch.Size([1, 3, 28, 28])
tensor([[-1.1529e+00,  5.9912e-01,  1.1856e+00,  2.0037e+00,  1.6274e+00,
          3.3721e+00,  9.2648e-01, -1.9575e+00, -1.5715e+00, -2.7994e+00,
         -9.4762e-01, -1.7412e+00, -1.1623e+00, -1.5199e+00, -5.1058e-01,
         -6.8135e-01, -2.6792e-01, -1.9036e+00, -5.2568e-01, -2.3722e+00,
         -2.0020e+00, -3.5556e-01, -4.4574e-01, -9.3151e-01, -8.4625e-01,
         -1.0598e+00,  6.3470e-01, -1.0059e+00, -7.8321e-01, -4.6957e-01,
         -1.3459e-01, -1.4642e+00, -6.9060e-01,  1.7900e+00,  1.3883e+00,
         -1.6197e+00,  1.4442e-01, -1.4012e+00,  6.4662e-01, -6.0506e-01,
         -1.1089e+00, -9.2006e-01, -1.0526e+00, -7.4314e-01, -7.0834e-01,
         -3.3165e-01, -1.8039e+00,  7.2059e-01, -2.7064e+00, -1.7809e-02,
          2.8596e-01, -1.7808e+00, -1.4320e+00,  7.4649e-02, -9.5867e-01,
         -1.1376e+00, -1.1169e+00, -9.9954e-01,  5.2636e-01, -1.2430e-01,
         -1.7088e+00, -3.3822e-01, -1.8691e+00,  1.5553e-01, -6.8265e-01,
          2

In [5]:
# Definição do dispositivo usado para processamento da rede. 'cuda' indica que será usada a GPU.
# Caso uma GPU com núcleos cuda (GPU NVIDIA) não esteja disponível, é necessário mudar o valor para 'cpu'.
# Nesse caso, o processamento será mais demorado.
DEVICE = 'cuda'                

def get_descriptors(imgs, model, bs, num_files, descriptor_size):
    '''Obtém descritores para as imagens do iterável `imgs`. O resultado da aplicação de `model`
    é usado como descritor. Note que seria mais apropriado realizar um treinamento adicional da rede
    para obtenção de descritores mais apropriados para o nosso problema. Mas esse é um assunto
    para o futuro...
    
    Parâmetros
    ----------
    imgs : iterável
        Iterável que retorna uma imagem e um rótulo a cada chamada 
    model : torch.nn.Module
        Rede neural do PyTorch
    bs : int
        Tamanho do batch
    num_files : int
        Número de elementos em `imgs`
    descriptor_size : int
        Número de descritores retornado por `model`
    '''
        
    model.eval()
    model.to(DEVICE)                                     # Coloca o modelo em DEVICE (usualmente a GPU)
    data = np.zeros((num_files, descriptor_size))
    labels = np.zeros(num_files, dtype=int)
    
    img, label = next(imgs)                                 # Verifica primeiro elemento para obter o tamanho da imagem
    img_shape = img.size                           
    batch = torch.zeros((bs, 3, img_shape[1], img_shape[0]))         
    batch[0] = transforms.ToTensor()(img.convert('RGB'))
    labels[0] = label
    
    # Vamos iterar sobre as imagens construindo batches contendo `bs` imagens. Quando um batch estiver
    # completo, aplicamos o modelo no batch
    for idx, (img, label) in enumerate(imgs, start=1):

        batch_idx = idx//bs
        idx_in_batch = idx - batch_idx*bs
        batch[idx_in_batch] = transforms.ToTensor()(img.convert('RGB'))
        labels[idx] = label

        if (idx+1)%bs==0:                         # Se completou um batch
            batch = batch.to(DEVICE)              # Move o batch para a GPU (se disponível)

            with torch.no_grad():                 # Esse comando indica que não iremos otimizar a rede
                output = model(batch)

            data[batch_idx*bs:(batch_idx+1)*bs] = np.array(output.to('cpu'))   # Salva os valores em um array 

            batch = torch.zeros_like(batch)                                    # Reseta o batch
            
    return data, labels

# Criamos o iterador novamente para incluir as imagens que já foram iteradas nas células acima
imgs = iter(image_generator(train_folder, files, filename_to_label))
data, labels = get_descriptors(imgs, model, 20, len(files), 1000)
print(data.shape)

(10000, 1000)


### Treinamento do classificador SVM usando os descritores

In [6]:
# Cria uma instância do classificador SVM. `C` é um parâmetro do classificador
# Essa célula pode demorar um pouco para rodar
svm = sklearn.svm.SVC(C=100)      
svm.fit(data, labels)              

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

### Reconhecimento de caracteres contidos em novas imagens

Vamos agora aplicar o nosso classificador em novas imagens para as quais não sabemos a classe. Para isso, para cada nova imagem precisamos obter os descritores e aplicar o classificador

In [7]:
def get_label(img, model, svm):
    '''Retorna a classe da imagem `img` usando os descritores de `model` e 
    o classificador `svm`'''
    
    img = transforms.ToTensor()(img.convert('RGB')).unsqueeze(0)
    img = img.to(DEVICE)
    with torch.no_grad(): 
        output = model(img)
    descriptor = np.array(output.to('cpu'))
    return svm.predict(descriptor)

# Carrega um novo conjunto de imagens
test_folder = root_folder+'teste/'
files = os.listdir(test_folder)
imgs = iter(image_generator(test_folder, files))

In [11]:
# Experimente rodar esta célula diversas vezes usando ctrl+enter
img, _ = next(imgs)
label = get_label(img, model, svm)
print(label)
img

[3]


## Verificação que uma camada convolucional é de fato uma operação de convolução

Vamos verificar se uma camada de convolução de uma rede neural realmente é uma operação de convolução

In [12]:
# Definição das variáveis

# Imagem
img = torch.tensor([[1, 3, 1, 2],
                    [0, 2, 1, 4],
                    [2, 1, 0, 1],
                    [3, 1, 2, 3]]).float()

# Filtro
w = torch.tensor([[1, 1, 1],
                  [1, 1, 1],
                  [1, 1, 1]]).float()

padding = 1
kernel_size = w.shape[0]

In [13]:
import torch.nn as nn      # nn é um submódulo possuindo diversos tipos de camadas de redes neurais

# Definição da camada de convolução. A camada possui 1 canal de entrada, 1 canal de saída (porque temos apenas 1 kernel) e 
# preenchimento com 0 para evitar problemas na borda. Por padrão, é somado um valor ao resultado da convolução, chamado 
# de bias, bias=False indica que não queremos somar esse valor.
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=kernel_size, padding=padding, 
                   bias=False, padding_mode='zeros')

# Por padrão, os valores do filtro da camada possuem valores aleatórios. Esse filtro é o atributo conv2d.weight.
# Esse atributo possui dimensão [1, 1, 3, 3], que é equivalente a um batch de tamanho 1, pois temos apenas
# 1 imagem na entrada, 1 filtro na camada e esse filtro possui tamanho 3x3
# Podemos mudar os valores de duas formas:

# Mudando os valores in-place
conv2d.weight[:] = w.view(1, 1, 3, 3)

# Criando um novo parâmetro e associando ao atributo
conv2d.weight = nn.Parameter(w.view(1, 1, 3, 3))

# A imagem precisa ter dimensão 4 ([tamanho do batch, num canais, num linhas, num colunas])
res_conv2d = conv2d(img.view(1, 1, 4, 4))
print(res_conv2d)

tensor([[[[ 6.,  8., 13.,  8.],
          [ 9., 11., 15.,  9.],
          [ 9., 12., 15., 11.],
          [ 7.,  9.,  8.,  6.]]]], grad_fn=<MkldnnConvolutionBackward>)
